In [59]:
import pandas as pd
import regex
import os
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
import gensim
from gensim.models.phrases import Phrases, Phraser
from ast import literal_eval
from nltk.tokenize import sent_tokenize, word_tokenize

In [80]:
path = "orignal dump"
df = pd.read_csv("Data.csv", index_col=0)

In [81]:
df.head()

,File,Season,SeasonID,Episode,EpisodeID,DialogueLines,Text,Word_count,Char_count,Stopwords
0,Suits.S01E01.srt,S01,1,E01,1,"[""Gerald Tate's here."", ""He wants to know what...",Gerald Tate's here. He wants to know what's ha...,9371,48607,3501
1,Suits.S01E02.srt,S01,1,E02,2,"['What is that, three in a row?', 'That would ...","What is that, three in a row? That would be fo...",6026,30980,2222
2,Suits.S01E03.srt,S01,1,E03,3,"[""♪ It's a perfect day ♪"", '♪ To go and tie on...",♪ It's a perfect day ♪ ♪ To go and tie one on ...,5843,30808,2150
3,Suits.S01E04.srt,S01,1,E04,4,"['Harvard trivia, the lightning round.', '[Sna...","Harvard trivia, the lightning round. [Snaps] G...",6469,34831,2307
4,Suits.S01E05.srt,S01,1,E05,5,"['Ross.', 'Triple double, courtesy of', ""Ameri...","Ross. Triple double, courtesy of America's fav...",5961,31022,2069


In [85]:
tokenized = []
for t in df["DialogueLines"]:
    ti = literal_eval(t)
    tokenized.extend(ti)
tokenized = [word_tokenize(t.replace("'","")) for t in tokenized]

In [86]:
tokenized[:10]

[['Gerald', 'Tates', 'here', '.'],
 ['He', 'wants', 'to', 'know', 'whats', 'happening', 'to', 'his', 'deal', '.'],
 ['Go', 'get', 'Harvey', '.'],
 ['I', 'check', '.'],
 ['Raise', '.'],
 ['Im', 'all-in', '.'],
 ['You', 'can', 'pay', 'me', 'later', '.'],
 ['I', 'got', 'to', 'go', '.'],
 ['Gentlemen', '.'],
 ['Im', 'paying', 'you', 'millions', ',', 'and', 'youre', 'telling', 'me']]

In [123]:
#to create the bigrams
bigram_model = Phrases(tokenized, delimiter=b"-")
bigram_sentences = []
bigram_sentences_t = []
#apply the trained model to a sentence
for t_sentence in tokenized:
    bigram_sentences.append(u' '.join(bigram_model[t_sentence]))
    bigram_sentences_t.append(bigram_model[t_sentence])
#get a trigram model out of the bigram
trigram_model = Phrases(bigram_sentences_t, delimiter=b"-")

trigram_sentences = []
trigram_sentences_t = []
#apply the trained model to a sentence
for t_sentence in bigram_sentences_t:
    trigram_sentences.append(u' '.join(trigram_model[t_sentence]))
    trigram_sentences_t.append(trigram_model[t_sentence])
#get a trigram model out of the bigram
fourgram_model = Phrases(trigram_sentences_t, delimiter=b"-")

In [71]:
df["BiGramPhraser"] = df["Text"].apply(lambda x : bigram_model[x]) 
df["TriGramPhraser"] = df["Text"].apply(lambda x : trigram_model[x]) 
df["FourGramPhraser"] = df["Text"].apply(lambda x : fourgram_model[x]) 

In [124]:
phrases = []
scores = []
for phrase, score in bigram_model.export_phrases(tokenized):
    phrases.append(phrase)
    scores.append(score)

for phrase, score in trigram_model.export_phrases(bigram_sentences_t):
    phrases.append(phrase)
    scores.append(score)

for phrase, score in fourgram_model.export_phrases(trigram_sentences_t):
    phrases.append(phrase)
    scores.append(score)

df_phrases = pd.DataFrame({"Phrase":phrases, "Score": scores, "Words": [len(p.split(b"-")) for p in phrases]})

In [125]:
df_phrases.sort_values(by=['Score'], inplace=True, ascending=False)

In [126]:
df_phrases= df_phrases.drop_duplicates()#head(20)

In [138]:
df_phrases[(df_phrases["Words"] == 3 & df_phrases["Phrase"].str.contains("[") == False)]

error: unterminated character set at position 0

In [122]:
df_phrases.shape

(1484, 3)